# Hoàng Phương Linh - 18020758

# Bài toán
Phân loại xem thú nuôi có được nhận nuôi với thời gian như thế nào kể từ khi đăng bài với 5 mốc ứng với 5 nhãn
* 0: Được nhận ngay trong ngày 
* 1: Được nhận trong khoảng thời gian 1 tuân (1 - 7 ngày)
* 2: Được nhận trong khoảng thời gian 1 tháng (8 - 30 ngày)
* 3: Được nhận trong khoảng thời gian 2 - 3 tháng (31 - 90 ngày)
* 4: Không được nhận trong 100 ngày đầu

# Dữ liệu
**Bảng**
- PetID - ID duy nhất của thú nuôi
- AdoptionSpeed - Tốc độ nhận nuôi, nhãn
- Type - Loại (1 = Chó, 2 = Mèo)
- Name - Tên
- Age - Tuổi, theo tháng
- Breed1 - Giống thứ nhất
- Breed2 - Giống thứ hai
- Gender - Giới tính (1 = Đực, 2 = Cái, 3 = Luỡng tính)
- Color1 - Màu thứ 1
- Color2 - Màu thứ  2 
- Color3 - Màu thứ  3 
- MaturitySize - Kích thuớc (1 = Nhỏ, 2 = Trung bình, 3 = Lớn, 4 = Rất lớn, 0 = Không biết)
- FurLength - Chiều dài (1 = Ngắn, 2 = Trung bình, 3 = Dài, 0 = Không biết)
- Vaccinated - Tiêm vaccin chưa (1 = Rồi, 2 = Chưa, 3 = Không biết)
- Dewormed - Bị sâu bệnh không (1 = Rồi, 2 = Chưa, 3 = Không biết)
- Sterilized - Thuần chủng hay không (1 = Rồi, 2 = Chưa, 3 = Không biết)
- Health - Sức khoẻ (1 = Khoẻ, 2 = Bị thưong nhẹ, 3 = Bị thương nặng, 0 = Không biết)
- Quantity - Số lượng thú nuôi được giới thiệu cùng
- Fee - Phí (0 = Miễn phí)
- State - Bang ở Malaysia
- RescuerID - ID của cơ sở cứu hộ
- VideoAmt - Số lượng video
- PhotoAmt - Số lượng ảnh
- Description - Miêu tả, có thể tiếng anh, Malay, Trung

**Ảnh**

**Image Metadata**
Dữ liệu lấy được dạng file json khi đưa ảnh qua Google's Vision API

**Sentiment Data**
Dữ liệu lấy được dạng file json khi đưa Description qua Google's Natural Language API

In [ ]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
import json
import os
import re
from sklearn import preprocessing
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms as T

In [ ]:
# load dữ liệu
train = pd.read_csv(r'../input/petfinder-adoption-prediction/train/train.csv')
test = pd.read_csv(r'../input/petfinder-adoption-prediction/test/test.csv')

# load sentiment đã đưọc lấy ra từ file json sentiment và save thành csv
sentiment = pd.read_csv(r'../input/sentiment-petfinder/sentiment.csv')
sentiment_test =  pd.read_csv(r'../input/sentiment-petfinder/sentiment_test.csv')

# load sentimentEntity đã đưọc lấy ra từ file json sentiment và save thành csv
sentimentEntity = pd.read_csv(r'../input/sentimententity-petfinder/sentimentEntity.csv').drop('Unnamed: 0', axis=1)
sentimentEntity_test = pd.read_csv(r'../input/sentimententity-petfinder/sentimentEntity_test.csv')

# load metadata đã đưọc lấy ra từ file json metadata và save thành csv
metadata = pd.read_csv(r'../input/metadata-petfinder/metadata.csv')
metadata_test = pd.read_csv(r'../input/metadata-petfinder/metadata_test.csv')

# Concat tất cả dữ liệu lại
train = pd.concat([train,sentiment,sentimentEntity,metadata], axis=1)
test = pd.concat([test,sentiment_test,sentimentEntity_test,metadata_test], axis=1)

* **Vấn đề:** Load các file json khá lâu
* **Cách làm:** Load file json rồi save thành file csv đẩy lên dataset rồi load ra mỗi khi cần
* **Kết quả:** Có các file đã được load sẵn thành csv tương ứng mỗi file là 1 dòng

# Feature

In [ ]:
f, ax = plt.subplots(figsize=(10,5))
plt.title("Số lượng đối với từng nhãn")
b1 = plt.bar([0,1,2,3,4], train['AdoptionSpeed'].value_counts().sort_index(),width=0.5)
plt.show()

Dữ liệu có chút không đều, thú nuôi được nhận ngay trong ngày rất ít, còn lại thì dữ liệu có vẻ đều hơn

**Type**

In [ ]:
def Type(data, istrain=True):
    if istrain==True:
        IsPureBreed0 = data[data['Type']==1]['AdoptionSpeed'].value_counts()/len(data[data['Type']==1])
        IsPureBreed1 = data[data['Type']==2]['AdoptionSpeed'].value_counts()/len(data[data['Type']==2])
        f, ax = plt.subplots(figsize=(10,5))
        plt.title("Chó hay mèo ứng với tốc độ nhận nuôi")
        b1 = plt.bar([4,3,2,1,0], IsPureBreed0,width=0.5)
        b2 = plt.bar([4.5,3.5,2.5,1.5,0.5], IsPureBreed1,width=0.5)
        plt.legend((b1[0], b2[0]), ('Chó', 'Mèo'))
        plt.show()
    return data
train = Type(train)

f, ax = plt.subplots(figsize=(10,5))
plt.title("Số lượng đối với từng loại")
labels = ['Chó', 'Mèo']
ax.set_xticks([1,2])
ax.set_xticklabels(labels)
b1 = plt.bar([1,2], train['Type'].value_counts().sort_index(),width=0.5)
plt.show()

Không có khác biệt quá đáng kể nhưng nhìn chung thì chó được nhận nuôi muộn hơn mèo

**Breed**

In [ ]:
def IsPureBreed(data, istrain=True):
    data['IsPureBreed'] = (data["Breed1"] == 0) | (data["Breed2"] == 0) | (data["Breed1"] == data["Breed2"])
    if istrain==True:
        IsPureBreed0 = data[data['IsPureBreed']==True]['AdoptionSpeed'].value_counts()/len(data[data['IsPureBreed']==True])
        IsPureBreed1 = data[data['IsPureBreed']!=True]['AdoptionSpeed'].value_counts()/len(data[data['IsPureBreed']!=True])
        f, ax = plt.subplots(figsize=(10,5))
        plt.title("Thuần chủng hay không thuẩn chủng ứng với tốc độ nhận nuôi")
        b1 = plt.bar([4,3,2,1,0], IsPureBreed0,width=0.5)
        b2 = plt.bar([4.5,3.5,2.5,1.5,0.5], IsPureBreed1,width=0.5)
        plt.legend((b1[0], b2[0]), ('Thuần chủng', 'Không thuần chủng'))
        plt.show()
    return data
train = IsPureBreed(train)

f, ax = plt.subplots(figsize=(10,5))
plt.title("Số lượng đối với thuần chủng hay không thuẩn chủng")
labels = ['Thuần chủng', 'Không thuần chủng']
ax.set_xticks([1,2])
ax.set_xticklabels(labels)
b1 = plt.bar([1,2], train['IsPureBreed'].value_counts().sort_index(),width=0.5)
plt.show()

Không thuần chủng thường được nhận nuôi sớm hơn thuần chủng

**Photo**

In [ ]:
def HavePhoto(data, istrain=True):
    data['HavePhoto'] = data['PhotoAmt']
    data.loc[data['HavePhoto']==0, 'HavePhoto'] = False
    data.loc[data['HavePhoto']!=0, 'HavePhoto'] = True
    if istrain == True:
        photo0 = data[data['PhotoAmt']==0]['AdoptionSpeed'].value_counts()/len(data[data['PhotoAmt']==0])
        photo1 = data[data['PhotoAmt']!=0]['AdoptionSpeed'].value_counts()/len(data[data['PhotoAmt']!=0])
        f, ax = plt.subplots(figsize=(10,5))
        plt.title("Có ảnh ứng với tốc độ nhận nuôi")
        b1 = plt.bar([4,3,2,1,0], photo0, width=0.5)
        b2 = plt.bar([4.5,3.5,2.5,1.5,0.5], photo1, width=0.5)
        plt.legend((b1[0], b2[0]), ('Không có ảnh', 'Có ảnh'))
        plt.show()
    return data
train = HavePhoto(train)

f, ax = plt.subplots(figsize=(10,5))
plt.title("Số lượng đối với có ảnh hay không có ảnh")
labels = ['Không có ảnh', 'Có ảnh']
ax.set_xticks([1,2])
ax.set_xticklabels(labels)
b1 = plt.bar([1,2], train['HavePhoto'].value_counts().sort_index(),width=0.5)
plt.show()

Không có ảnh sẽ được nhận nuôi muộn hơn nhiều so với có ảnh

**NumberColor**

In [ ]:
def NumberColor(data, istrain=True):
    temp = ((data['Color1']==0) & (data['Color2']==0))|((data['Color1']==0) & (data['Color3']==0))|((data['Color3']==0) & (data['Color2']==0))
    data['NumberColor'] = [1 if (tmp == True) else 0 for tmp in temp]
    temp = ((data['Color1']!=0) & (data['Color2']!=0) & (data['Color3']==0))|((data['Color1']!=0) & (data['Color3']!=0)& (data['Color2']==0))|((data['Color3']!=0) & (data['Color2']!=0) & (data['Color1']==0))
    data.loc[[tmp == True for tmp in temp], 'NumberColor'] = 2
    temp = ((data['Color1']!=0) & (data['Color2']!=0) & (data['Color3']!=0))
    data.loc[[tmp == True for tmp in temp], 'NumberColor'] = 3
    if istrain == True:
        d1 = data[data['NumberColor']==1]['AdoptionSpeed'].value_counts()/len(data[data['NumberColor']==1])
        d2 = data[data['NumberColor']==2]['AdoptionSpeed'].value_counts()/len(data[data['NumberColor']==2])
        d3 = data[data['NumberColor']==3]['AdoptionSpeed'].value_counts()/len(data[data['NumberColor']==3])
        f, ax = plt.subplots(figsize=(10,5))
        plt.title("Số luợng màu sắc trên cơ thể ứng với tốc độ nhận nuôi")
        b1 = plt.bar([6,4.5,3,1.5,0], d1,width=0.5)
        b2 = plt.bar([6.5,5,3.5,2,0.5], d2,width=0.5)
        b3 = plt.bar([7,5.5,4,2.5,1], d3,width=0.5)
        labels = [4,3,2,1,0]
        ax.set_xticks([6.5,5,3.5,2,0.5])
        ax.set_xticklabels(labels)
        plt.legend((b1[0], b2[0], b3[0]), ('1 màu', '2 màu', '3 màu'))
        plt.show()
    return data
train = NumberColor(train)

f, ax = plt.subplots(figsize=(10,5))
plt.title("Số lượng đối với từng màu")
labels = ['1 màu', '2 màu', '3 màu']
ax.set_xticks([1,2,3])
ax.set_xticklabels(labels)
b1 = plt.bar([1,2,3], train['NumberColor'].value_counts().sort_index(),width=0.5)
plt.show()

Tỷ lệ khá gần nhau nhưng vật nuôi có 1 màu lại có tỷ lệ nhận nuôi sau 100 ngày lớn hơn

**OnePet**

In [ ]:
def OnePet(data, istrain=True):
    data['onePet'] = (data['Quantity']==1)
    if istrain == True:
        onePet1 = data[data['onePet']==True]['AdoptionSpeed'].value_counts()/len(data[data['onePet']==True])
        onePet0 = data[data['onePet']!=True]['AdoptionSpeed'].value_counts()/len(data[data['onePet']!=True])
        f, ax = plt.subplots(figsize=(10,5))
        plt.title("Bài post có 1 hay nhiều thú nuôi ứng với tốc độ nhận nuôi")
        b1 = plt.bar([4,3,2,1,0], onePet1,width=0.5)
        b2 = plt.bar([4.5,3.5,2.5,1.5,0.5], onePet0,width=0.5)
        plt.legend((b1[0], b2[0]), ('1 con', 'nhiều con'))
        plt.show()
    return data
train = OnePet(train)

f, ax = plt.subplots(figsize=(10,5))
plt.title("Số lượng đối với bài post có 1 hay nhiều thú nuôi")
labels = ['1 con', 'nhiều con']
ax.set_xticks([1,2])
ax.set_xticklabels(labels)
b1 = plt.bar([2,1], train['onePet'].value_counts().sort_index(),width=0.5)
plt.show()

Thường thú nuôi trong post có nhiều con sẽ được nhận nuôi muộn hơn

**HaveVideo**

In [ ]:
def HaveVideo(data, istrain=True):
    data['HaveVideo'] = (data['VideoAmt']!=0)
    if istrain == True:
        o1 = data[data['HaveVideo']==True]['AdoptionSpeed'].value_counts()/len(data[data['HaveVideo']==True])
        o0 = data[data['HaveVideo']!=True]['AdoptionSpeed'].value_counts()/len(data[data['HaveVideo']!=True])
        f, ax = plt.subplots(figsize=(10,5))
        plt.title("Có video hay không ứng với tốc độ nhận nuôi")
        b1 = plt.bar([4,3,2,1,0], o1,width=0.5)
        b2 = plt.bar([4.5,3.5,2.5,1.5,0.5], o0,width=0.5)
        plt.legend((b1[0], b2[0]), ('Có video', 'Không có video'))
        plt.show()
    return data
train = HaveVideo(train)

f, ax = plt.subplots(figsize=(10,5))
plt.title("Số lượng đối với có video hay không có video")
labels = ['Không có video', 'Có video']
ax.set_xticks([1,2])
ax.set_xticklabels(labels)
b1 = plt.bar([1,2], train['HaveVideo'].value_counts().sort_index(),width=0.5)
plt.show()

**Sentiment**

In [ ]:
def Sentiment(data, istrain=True):
    threshold1 = 0.6
    threshold2 = - threshold1
    thOfMagnitude = 3

    Positive = 5
    Negative = -5
    Neutral = 0
    Mixed = 2
    temp = []
    for s, m in zip(data['SentimentScore'],data['SentimentMagnitude']):
        if s >= threshold1:
            temp.append(Positive)
        elif s > threshold2 and s < threshold1 and m >= thOfMagnitude:
            temp.append(Mixed)
        elif s > threshold2 and s < threshold1 and m < thOfMagnitude:
            temp.append(Neutral)
        elif s <= threshold2:
            temp.append(Negative)

    data['Sentiment'] = temp
    if istrain == True:
        d1 = data[data['Sentiment']==Positive]['AdoptionSpeed'].value_counts()/len(data[data['Sentiment']==Positive])
        d2 = data[data['Sentiment']==Mixed]['AdoptionSpeed'].value_counts()/len(data[data['Sentiment']==Mixed])
        d3 = data[data['Sentiment']==Neutral]['AdoptionSpeed'].value_counts()/len(data[data['Sentiment']==Neutral])
        d4 = data[data['Sentiment']==Negative]['AdoptionSpeed'].value_counts()/len(data[data['Sentiment']==Negative])
        f, ax = plt.subplots(figsize=(10,5))
        plt.title("Cảm xúc ứng với tốc độ nhận nuôi")
        b1 = plt.bar([8,6,4,2,0], d1,width=0.5)
        b2 = plt.bar([8.5, 6.5, 4.5, 2.5, 0.5], d2,width=0.5)
        b3 = plt.bar([9, 7, 5, 3, 1], d3,width=0.5)
        b4 = plt.bar([9.5, 7.5, 5.5, 3.5, 1.5], d3,width=0.5)
        plt.legend((b1[0], b2[0], b3[0], b4[0]), ('Tích cực', 'Lẫn lộn', 'Trung lập', "Tiêu cực"))
        plt.show()
    return data
train = Sentiment(train)

f, ax = plt.subplots(figsize=(10,5))
plt.title("Số lượng đối với từng cảm xúc")
labels = ['Tích cực', 'Lẫn lộn', 'Trung lập', "Tiêu cực"]
ax.set_xticks([1,2,3,4])
ax.set_xticklabels(labels)
b1 = plt.bar([4,3,2,1], train['Sentiment'].value_counts().sort_index(),width=0.5)
plt.show()

Cảm xúc tích cực có thời gian nhận nuôi muộn hơn 100 ngày nhiều nhất với các cảm xúc còn lại